In [ ]:
# этот скрипт недоделан.

# этот скрипт подбирает оптимальный стоплосс для рандомного трейда.

In [145]:
#%pip install numpy
#%pip install pandas
#%pip install mplfinance
#%pip install matplotlib
#%pip install ta
#%pip install pandas_ta

In [146]:
import numpy as np
import pandas as pd
import mplfinance as mpf
import matplotlib.pyplot as plt
#import ta # https://github.com/bukosabino/ta
import pandas_ta as ta

In [147]:
# параметры:
symbol = 'BTCUSD'
charts_at = '2024-05-29 08:10' # время окончания графика

In [149]:
def get_df(date_time, period, length = 1000):
    """
    Загрузить датафрейм с диска и взять с него слайс.
    
    Параметры:
    date_time : string
        Например, 2024-05-29 08:10. Время не обязательно должно попадать в точку графика (будет взята ближайшая точка).
    period : string
        Допустимые значения: '5S' - 5 секунд, '1T' - 1 minute, '5T' - 5 minutes, '1H' - 1 hour, 'D' - 1 day.
    length : number
        Сколько точек надо в датафрейме.
    """
    if period == '5S': # пятисекундные свечи
        # '2024-05-29 08:10' -> '2024-05-29'
        date = date_time[:10]
        filename = f'npz/{symbol}/{symbol}{date}.npz'
    else:
        filename = f'npz/{symbol}_{period}.npz'

    # зачитывание в формате numpy
    data = np.load(filename, allow_pickle=True)['data']
    # преобразование в формат pandas
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

    # Преобразование 'timestamp' в datetime и установка его как индекс DataFrame
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)

    # Преобразование всех столбцов с ценами и объемом в числовой формат
    for column in ['open', 'high', 'low', 'close', 'volume']:
        df[column] = pd.to_numeric(df[column], errors='coerce')

    # поиск интересной точки.
    date_to_find = date_to_find = pd.Timestamp(date_time)
    index_position = df.index.get_indexer([date_to_find], method='nearest')[0]
    # беру часть точек, чтобы не напрягать либу технического анализа большим количеством точек.
    df = df[index_position-length:index_position]
    return df

In [150]:
df = get_df(charts_at, '1T')